In [1]:
!pip install -q kaggle

# Upload your kaggle.json file here
from google.colab import files
files.upload()  # Upload kaggle.json

# Move kaggle.json to ~/.kaggle and set permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d surajkarakulath/labelled-corpus-political-bias-hugging-face
!unzip -q labelled-corpus-political-bias-hugging-face.zip


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/surajkarakulath/labelled-corpus-political-bias-hugging-face
License(s): other
  0% 0.00/47.8M [00:00<?, ?B/s]
100% 47.8M/47.8M [00:00<00:00, 1.35GB/s]


In [2]:
!du -sh *


30M	Center Data
4.0K	kaggle.json
48M	labelled-corpus-political-bias-hugging-face.zip
73M	Left Data
36M	Right Data
55M	sample_data


In [3]:
!pip install -q transformers datasets scikit-learn pandas tqdm


In [4]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [1]:
import os
import pandas as pd

def load_data(folder_path, label):
    data = []
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read().strip()
            if text:
                data.append((text, label))
    return data

# Load data
left_data = load_data('Left Data/Left Data', 0)
center_data = load_data('Center Data/Center Data', 1)
right_data = load_data('Right Data/Right Data', 2)

# Combine into a dataframe
all_data = pd.DataFrame(left_data + center_data + right_data, columns=['text', 'label'])
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle
print(all_data['label'].value_counts())
all_data.head()


label
0    7803
2    5563
1    3996
Name: count, dtype: int64


,text,label
0,Oregon Gov. Kate Brown said Wednesday that the...,0
1,Credit...Richard Perry/The New York TimesMarch...,0
2,An independent panel of medical experts overse...,2
3,PoliticsBiden Keeps Up Pressure On Putin To Pu...,0
4,"| December 30, 2020 09:39 AM In...",2


In [2]:
from sklearn.model_selection import train_test_split

# Split into train/val/test
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    all_data['text'], all_data['label'], test_size=0.3, random_state=42, stratify=all_data['label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)


In [3]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [4]:
import torch

class BiasDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = BiasDataset(train_encodings, list(train_labels))
val_dataset = BiasDataset(val_encodings, list(val_labels))
test_dataset = BiasDataset(test_encodings, list(test_labels))


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
output_dir = "/content/drive/MyDrive/political_bias_model"


In [7]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

training_args = TrainingArguments(
    output_dir=output_dir,  # Save all checkpoints, logs, model here
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir=f"{output_dir}/logs",  # Log files
    save_total_limit=2,  # Keep only the 2 most recent checkpoints to save space
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False  # Because lower eval_loss is better
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import os
os.makedirs(output_dir, exist_ok=True)


In [10]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)
    labels = p.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1_macro': f1_score(labels, preds, average='macro'),
        'f1_weighted': f1_score(labels, preds, average='weighted'),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: manav1102004 (manav1102004-thapar-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,0.730200,0.466112,0.800691,0.794974,0.801171
2,0.379700,0.390551,0.849462,0.842429,0.848648
3,0.284800,0.435843,0.846774,0.840276,0.845825
4,0.198100,0.455188,0.855607,0.849164,0.855076


TrainOutput(global_step=3040, training_loss=0.37799809857418665, metrics={'train_runtime': 3941.4346, 'train_samples_per_second': 12.334, 'train_steps_per_second': 0.771, 'total_flos': 6439620022898688.0, 'train_loss': 0.37799809857418665, 'epoch': 4.0})

In [12]:
results = trainer.evaluate(test_dataset)
print(results)


{'eval_loss': 0.395058810710907, 'eval_accuracy': 0.8483685220729367, 'eval_f1_macro': 0.8413727924054278, 'eval_f1_weighted': 0.8472394361139632, 'eval_runtime': 56.1183, 'eval_samples_per_second': 46.42, 'eval_steps_per_second': 0.731, 'epoch': 4.0}


In [14]:
# Save model + tokenizer to Drive
model_path = "/content/drive/MyDrive/political_bias_model/final_model"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('/content/drive/MyDrive/political_bias_model/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/political_bias_model/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/political_bias_model/final_model/vocab.txt',
 '/content/drive/MyDrive/political_bias_model/final_model/added_tokens.json',
 '/content/drive/MyDrive/political_bias_model/final_model/tokenizer.json')

In [15]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

model_path = "/content/drive/MyDrive/political_bias_model/final_model"

tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()  # Important for inference


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [16]:
label_map = {0: "Left", 1: "Center", 2: "Right"}


In [17]:
import pandas as pd
import random

# Create DataFrame again if not already done
all_data = pd.DataFrame(left_data + center_data + right_data, columns=['text', 'label'])

# Recreate test split (same as before)
_, temp_texts, _, temp_labels = train_test_split(
    all_data['text'], all_data['label'], test_size=0.3, random_state=42, stratify=all_data['label']
)
_, test_texts, _, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Pick 10 random samples
sample_indices = random.sample(range(len(test_texts)), 10)
sample_texts = test_texts.iloc[sample_indices].tolist()
sample_labels = test_labels.iloc[sample_indices].tolist()


In [18]:
for i, text in enumerate(sample_texts):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=1).item()

    print(f"Sample #{i+1}")
    print(f"📜 Text (truncated): {text[:200]}...")
    print(f"✅ Predicted Bias: {label_map[pred]}")
    print(f"🎯 Actual Bias   : {label_map[sample_labels[i]]}")
    print("-" * 80)


Sample #1
📜 Text (truncated): Roger Ailes, who spent decades advising political candidates and presidents about the power of television in American life before building Fox News into a media juggernaut ― and who left that network ...
✅ Predicted Bias: Left
🎯 Actual Bias   : Left
--------------------------------------------------------------------------------
Sample #2
📜 Text (truncated): Story highlightsMike Huckabee called Friday's shootings at a Planned Parenthood clinic 'domestic terrorism'Huckabee also blasted President Barack Obama for the tone of his remarks about ISISThe former...
✅ Predicted Bias: Left
🎯 Actual Bias   : Left
--------------------------------------------------------------------------------
Sample #3
📜 Text (truncated): Job growth slowed dramatically in March.   Hiring slowed sharply in March, with the economy adding only 88,000 jobs, the lowest monthly gain since last June. Economists surveyed by CNNMoney were expec...
✅ Predicted Bias: Left
🎯 Actual Bias   : Lef

In [19]:
generated_articles = [
    # LEFT-leaning samples
    "The government's crackdown on climate activists has raised serious concerns among civil liberties groups. Environmental advocates say the administration is silencing dissent.",

    "With income inequality at historic highs, progressive lawmakers are pushing for higher taxes on the wealthy and expanded social programs to support working-class Americans.",

    # RIGHT-leaning samples
    "The Senate has passed a bipartisan infrastructure bill with support from both Republicans and Democrats. Analysts say this could mark a return to cooperative policymaking.",

    "Experts are urging caution on foreign policy moves, emphasizing diplomacy and strategic alliances over unilateral action. Both sides of the aisle support a measured approach.",

    "A new wave of conservative leaders are calling for stronger border security measures, arguing that the current administration has failed to enforce immigration law.",

    "Gun rights advocates are rallying nationwide in response to proposed firearm restrictions. They argue the Second Amendment is under threat from progressive lawmakers.",

    "The Supreme Court's recent ruling on free speech is being hailed as a victory for constitutional rights, though opinions are split across party lines.",

    "Unemployment rates dropped to pre-pandemic levels, according to the latest Labor Department report. Economists credit the rebound to a mix of private sector growth and federal stimulus.",

    # LEFT-leaning samples
    "President’s approval rating has fallen amid rising inflation and geopolitical tensions, with critics from both left and right questioning the administration’s priorities.",

    "Social media platforms face scrutiny from lawmakers concerned about misinformation. Regulatory proposals vary widely across the political spectrum."
]


In [20]:
for i, text in enumerate(generated_articles):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    pred_label = torch.argmax(outputs.logits, dim=1).item()

    print(f"📰 Article #{i+1}")
    print(f"📜 Text: {text}")
    print(f"🔍 Predicted Bias: {label_map[pred_label]}")
    print("-" * 100)


📰 Article #1
📜 Text: The government's crackdown on climate activists has raised serious concerns among civil liberties groups. Environmental advocates say the administration is silencing dissent.
🔍 Predicted Bias: Left
----------------------------------------------------------------------------------------------------
📰 Article #2
📜 Text: With income inequality at historic highs, progressive lawmakers are pushing for higher taxes on the wealthy and expanded social programs to support working-class Americans.
🔍 Predicted Bias: Right
----------------------------------------------------------------------------------------------------
📰 Article #3
📜 Text: The Senate has passed a bipartisan infrastructure bill with support from both Republicans and Democrats. Analysts say this could mark a return to cooperative policymaking.
🔍 Predicted Bias: Right
----------------------------------------------------------------------------------------------------
📰 Article #4
📜 Text: Experts are urging cau

In [22]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

model_path = "/content/drive/MyDrive/political_bias_model/final_model"

tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
